In [0]:
import torch
import numpy as np
import logging

In [0]:
def find_center(trainloader=None, classifier=None, rep_dim=None, device='cuda', eps=0.1):
    """Initialize hypersphere center c as the mean from an initial forward pass on the data."""
    if rep_dim is None:
        raise ValueError('Specify rep_dim to find center.')
    
    classifier = classifier.to(device)    
    n_samples = 0
    c = torch.zeros(rep_dim, device=device)
    logger = logging.getLogger()
    
    logger.info('Initializing Center...')

    train_mapped = []
    classifier.eval()
    with torch.no_grad():
        for data in trainloader:
            # get the inputs of the batch
            inputs, _, _ = data
            inputs = inputs.to(device)
            outputs = classifier(inputs)
            train_mapped.extend(outputs.tolist())
            n_samples += outputs.shape[0]
            c += torch.sum(outputs, dim=0)

    c /= n_samples
    train_mapped = np.array(train_mapped)

    # If c_i is too close to 0, set to +-eps. Reason: a zero unit can be trivially matched with zero weights.
    c[(abs(c) < eps) & (c < 0)] = -eps
    c[(abs(c) < eps) & (c > 0)] = eps
    
    logger.info('Center initialized.')

    return c